# Download datasets

In [9]:
import os
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import openpyxl
from bs4 import BeautifulSoup
from pathlib import Path
from openpyxl.utils.dataframe import dataframe_to_rows

## Fetching data

A script was created to download all datasets, unzip and put them in their corresponding ```data``` folder.

In [37]:
import zipfile, urllib.request, shutil
import time
from pathlib import Path
from urllib.parse import urlparse

In [39]:
cwd = Path.cwd()


def CreateDir(path):
    if not os.path.exists(path):
        os.makedirs(path)

#downloads zip file from an url and extracts it to a folder
def DownloadDatasets(url, file_name, dir_name):
    try:
        with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)        
            with zipfile.ZipFile(file_name,mode='r') as zf:
                zf.extractall(dir_name)
        time.sleep(3)
        #move zip to dir_name
        shutil.move(file_name, dir_name)
    except Exception:
        print ("Something happened with zip file ",file_name)
        pass     

def DownloadFrom(base_url, area, start_year, end_year):
    UrlsByYear = [str(i) for i in range(start_year,end_year)]
    UrlsByYear = [s[2:]  for s in UrlsByYear]
    UrlsByYear = [base_url + s + area + ".zip" for s in UrlsByYear] 
    for url in UrlsByYear:
        print("Getting data from: ", url)
        infoUrl = urlparse(url)
        file_name = os.path.basename(infoUrl.path)
        dir_name = str(cwd) + "/" + area + "/" + str(os.path.splitext(file_name)[0])
        CreateDir(dir_name)
        DownloadDatasets(url, file_name, dir_name)
        

#DownloadFrom("http://datosabiertos.aire.cdmx.gob.mx:8080/opendata/excel/RAMA/","RAMA",1986, 2021)
#DownloadFrom("http://datosabiertos.aire.cdmx.gob.mx:8080/opendata/excel/REDMET/","REDMET",1986,2021)
#DownloadFrom("http://datosabiertos.aire.cdmx.gob.mx:8080/opendata/excel/REDMA/","REDMA",1986,2021)
#DownloadFrom("http://datosabiertos.aire.cdmx.gob.mx:8080/opendata/excel/REDDA/","REDDA",1988,2020)
#DownloadFrom("http://datosabiertos.aire.cdmx.gob.mx:8080/opendata/excel/RADIACION/","RADIACION",2000,2021)  #EL AÑO 2020 NO EXISTE EN LA PAG
#DownloadFrom("http://datosabiertos.aire.cdmx.gob.mx:8080/opendata/excel/PRESION/","PRESION",2009,2021)


### Dataset consolidation:

To have a consolidated dataset for each category we propose the following method:
1. For each category folder: **PRESION, RADIACION, RAMA, REDDA, REDMA and REDMET** we loop all its directories (organized by year in the range ```1986-2020``` and containing Factor data. Read those files that have extension ```.xls```.
2. For each ```excel factor file``` having data for Factors ```CO2, NO2, NOX, 03, PM10, SO2, UVM, etc.```. Load a dataframe from excel data, then add corresponding  ```Factor``` label column to characterize the data. 
3. Loop all dataframes to get all columns that must be present in **condolidated** dataset. 
4. Check if current dataframe has all general columns, if that is not the case we add a column with null values repreented by ```NaNfile``` **-99**. 
5. Merge all dataframes and save it to a new file. Result of this procedure is a ```single dataset per year```.
6. Finally we repeat the proces using all generated year files to create a single dataset per category.


<img src="img/sampleYearResult.png" >
<img src="img/SampleConsolidated.png" >


In [ ]:
import os
cwd = os.getcwd()
datad = cwd + '\\data\\aire.cdmx\\'
nullValue = -99
slash = '\\'

#returns a set of all columns that must be in all files            
def GetAllColumnsFromFiles(yearDir): 
    Columns = set()
    for FactorFile in os.listdir(yearDir):
        if FactorFile.endswith('.xls') or FactorFile.endswith('.xlsx'):
            xl = pd.ExcelFile(yearDir+FactorFile)            
            df = xl.parse(xl.sheet_names[0])
            df.columns = df.columns.str.upper()
            for c in df.columns:
                Columns.add(c.upper())
            #print("File columns: ", df.columns)
            #print("number of columns for file: ",len(df.columns))
    #print("All columns for directory: ", Columns)
    #print("Total columns for year dataset to use:", len(Columns))
    return list(Columns) 

#adds a column with a certain value to a dataframe
def AddColumnToDataFrame(df, columnName, value):
    df[columnName] = value

#removes starting year and extension: radiacion_2000UVA.xls,  1986O2.xls etc
def GetFactorFromFileName(s):
    i = 0
    s = s.replace("_","")
    for c in s:
        if c.isalpha():
            i += 1
        else:
            break
    return s[4+i:-4]

#Adds missing column and fill it with NaN, then adds a Factor label column 
def ManageFactorColumnsForDataFrame(temp_df,Columns,FactorFile, addFactorColumn):
    for c in Columns:
        if c.upper() not in temp_df.columns:
            AddColumnToDataFrame(temp_df,c, nullValue)  
    if addFactorColumn:        
        AddColumnToDataFrame(temp_df, "FACTOR", GetFactorFromFileName(FactorFile) ) 

    
def SaveCsv(df, path, csvFileName):
    p = os.path.join( str(Path(path).parents[0]) + slash + csvFileName ) + ".csv"
    #print("saving to: ", p )
    df.to_csv( p, sep=',', encoding='utf-8', header='true', index=False)

    
def SaveXls(df, path, xlsFileName, compress=False):
    p = os.path.join( str(Path(path).parents[0]) + slash + xlsFileName ) + ".xlsx"
    #print("saving to: ", p )
   
    writer = pd.ExcelWriter(p, engine='xlsxwriter')
    if compress:
        writer.book.use_zip64()
    df.to_excel(writer, sheet_name=xlsFileName, index=False)
    writer.save()
    
    '''wb = openpyxl.Workbook()
    ws = wb.active
    for r in dataframe_to_rows(df, index=False, header=True):
        if len(r) > 1:
            ws.append(r)
    wb.save(p)  
    '''
    
def ConsolidateExcelFiles(workspaceDir, resultFileName, addFactorColumn =True, resultIsXls = True):
    #print("Starting directory: ", workspaceDir)
    dfs = []
    Columns = GetAllColumnsFromFiles(workspaceDir)
       
    for excelFile in os.listdir(workspaceDir):
        if excelFile.endswith('.xls') or excelFile.endswith('.xlsx'):
            xl = pd.ExcelFile(workspaceDir+excelFile)            
            temp_df = xl.parse(xl.sheet_names[0])
            temp_df.columns = temp_df.columns.str.upper()
            if not temp_df.empty:
                ManageFactorColumnsForDataFrame(temp_df, Columns, excelFile,addFactorColumn)
                dfs.append(temp_df)
            else:
                print("There is an issue with file: ", excelFile)
            print("Finished parsing file:", excelFile) 
    
    merged_df = pd.concat(dfs)
    
    merged_df["FECHA"] = pd.to_datetime(merged_df["FECHA"])
    
    #merged_df = merged_df.sort_values(['FECHA', 'HORA', 'FACTOR'], ascending=[True, True])    
    if resultIsXls :
        #compress = True if df.axes[0] > 1000000 else False
        if len(merged_df.axes[0]) < 1000000:
            SaveXls(merged_df, workspaceDir, resultFileName ) 
        else: 
            print("Dataser is to large for excel, saving as csv, no. rows:", len(merged_df.axes[0]))
            SaveCsv(merged_df, workspaceDir, resultFileName)    
    else:
        SaveCsv(merged_df, workspaceDir, resultFileName) 
    print("---------------------------Finished consolidating file:", resultFileName)
    

In [40]:
def ConsolidateByYear(dirToConsolidate):
    for yearFile in os.listdir(dirToConsolidate):
        path = dirToConsolidate + yearFile + slash
        if os.path.isdir(path): 
            ConsolidateExcelFiles(path,yearFile, True, True)
#main
'''
for file in os.listdir(datad):
    if os.path.isdir(datad+file): 
        path = os.path.join(datad+file+slash)
        ConsolidateByYear(path)
'''           

workDir ="C:\\Users\\Edgar\\Desktop\\DS4A\\Project\\data\\aire.cdmx\\RAMA\\"
#ConsolidateByYear(workDir)
ConsolidateExcelFiles(workDir, "RAMA",  False, True )



Finished parsing file: 00RAMA.xlsx
Finished parsing file: 01RAMA.xlsx
Finished parsing file: 02RAMA.xlsx
Finished parsing file: 03RAMA.xlsx
Finished parsing file: 04RAMA.xlsx
Finished parsing file: 05RAMA.xlsx
Finished parsing file: 06RAMA.xlsx
Finished parsing file: 07RAMA.xlsx
Finished parsing file: 08RAMA.xlsx
Finished parsing file: 09RAMA.xlsx
Finished parsing file: 10RAMA.xlsx
Finished parsing file: 11RAMA.xlsx
Finished parsing file: 12RAMA.xlsx
Finished parsing file: 13RAMA.xlsx
Finished parsing file: 14RAMA.xlsx
Finished parsing file: 15RAMA.xlsx
Finished parsing file: 16RAMA.xlsx
Finished parsing file: 17RAMA.xlsx
Finished parsing file: 18RAMA.xlsx
Finished parsing file: 19RAMA.xlsx
Finished parsing file: 20RAMA.xlsx
Finished parsing file: 86RAMA.xlsx
Finished parsing file: 87RAMA.xlsx
Finished parsing file: 88RAMA.xlsx
Finished parsing file: 89RAMA.xlsx
Finished parsing file: 90RAMA.xlsx
Finished parsing file: 91RAMA.xlsx
Finished parsing file: 92RAMA.xlsx
Finished parsing fil